# 导入相关库

In [91]:
import os
import pathlib
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pickle as pkl

from time import time
from tqdm.notebook import tqdm

# 数据获取和处理

In [76]:
# 数据目录下保存了很多csv文件，每个csv文件保存了一个5s的sequence，包含了多个车辆的轨迹信息
DATASET_PATH = "/ssd/datasets/argoverse/argoverse-forecasting-dataset/"
TRAIN = DATASET_PATH + "train/data"
VAL = DATASET_PATH + "val/data"
TEST = DATASET_PATH + "test_obs/data/"

def data_process(root_dir, mode):
    """
    处理root_dir目录下的CSV文件
    返回numpy array, shape: (total num of sequences, seq len, 4)
    """
    root_dir = pathlib.Path(root_dir)
    paths = [(root_dir / filename).absolute() for filename in os.listdir(root_dir)]
    seq_len = 50 if mode != "test" else 20
    features = np.empty((len(paths), seq_len, 4))
    for i in tqdm((range(len(paths)))):
        path = paths[i]
        sequence = pd.read_csv(path)
        agent_x = sequence[sequence["OBJECT_TYPE"] == "AGENT"]["X"]
        agent_y = sequence[sequence["OBJECT_TYPE"] == "AGENT"]["Y"]
        xy = np.column_stack((agent_x, agent_y))
        # 如果是train或者val，则xy shape: (50, 2) 记录了5秒（每秒10帧）的agent xy坐标
        # 否则xy shape: (20, 2)，是agent前2s的xy坐标
        vel = xy[1:] - xy[:-1]
        init_unknown_vel = np.array([np.nan, np.nan])
        vel = np.vstack((init_unknown_vel, vel))
        # vel shape: (seq len, 2)，差分得到速度，初始速度无法获取，设为NaN
        feature = np.column_stack((xy, vel))
        # feature shape: (seq len, 4), 各列分别是x, y, vel_x, vel_y
        features[i] = feature
    return features

def save_features_to_pkl(features, filepath):
    basedir = os.path.dirname(filepath)
    if not os.path.exists(basedir):
        os.makedirs(basedir)
    with open(filepath, 'wb') as f:
        pkl.dump(features, f)
        
def load_pkl_to_features(filepath):
    with open(filepath, 'rb') as f:
        features = pkl.load(f)
    return features

In [69]:
d = {"train": TRAIN,
     "val": VAL,
     "test": TEST} 
for mode, path in tqdm(d.items()):
    features = data_process(path, mode)
    save_path = 'data/{}.pkl'.format(mode)
    save_features_to_pkl(features, save_path)

# 为Pytorch模型加载数据创建接口

In [82]:
class TrajectoryDataset(Dataset):
    
    def __init__(self, pkl_filepath, mode, obs_len = 20):
        self.mode = mode
        self.features = load_pkl_to_features(pkl_filepath)
        self.obs_len = obs_len
        
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, idx):
        # 返回source以及target
        return [self.features[idx, :self.obs_len], self.features[idx, self.obs_len:]]
    
class WrappedDataLoader:
    def __init__(self, dataloader, func):
        self.dataloader = dataloader
        self.func = func
        
    def __len__(self):
        return len(self.dataloader)
    
    def __iter__(self):
        iter_dataloader = iter(self.dataloader)
        for batch in iter_dataloader:
            yield self.func(*batch)

def batch_second(x, y):
    """
    输入: x = [batch size, 20, 4], y = [batch size, 30, 4]
    我们将batch size 放到第二个维度，从而使
    x = [20, batch size, 4]
    y = [30, batch size, 4]
    """
    return x.transpose(0, 1), y.transpose(0, 1)

In [83]:
train_dataset = TrajectoryDataset("data/train.pkl", "train")
val_dataset = TrajectoryDataset("data/val.pkl", "val")
test_dataset = TrajectoryDataset("data/test.pkl", "test")

BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6)
train_loader = WrappedDataLoader(train_loader, batch_second)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE * 2, shuffle=False, num_workers=6)
val_loader = WrappedDataLoader(val_loader, batch_second)

In [85]:
for x, y in train_loader:
    print(x.shape, y.shape)
    print(x[:2, 0, :])
    break

torch.Size([20, 128, 4]) torch.Size([30, 128, 4])
tensor([[ 7.3972e+02,  2.2534e+03,         nan,         nan],
        [ 7.3963e+02,  2.2529e+03, -8.7988e-02, -5.7026e-01]],
       dtype=torch.float64)


# 定义Transformer模型

In [93]:
class TrajectoryTransformer(nn.Module):
    def __init__(self,
                 device,
                 source_seq_len: int = 19,
                 target_seq_len: int = 30,
                 input_dim: int = 2,
                 output_dim: int = 2,
                 d_model: int = 512,
                 nhead: int = 8,
                 num_encoder_layers: int = 6,
                 num_decoder_layers: int = 6,
                 dim_feedforward: int = 2048,
                 dropout: float = 0.1,
                 activation: str = 'relu'):
        super().__init__()
        self.source_seq_len = source_seq_len
        self.target_seq_len = target_seq_len
        self.total_seq_len = self.source_seq_len + self.target_seq_len
        self.device = device
        self.scale = torch.sqrt(torch.FloatTensor([d_model])).to(device)
        
        self.encoder_embedding = nn.Linear(input_dim, d_model)
        self.decoder_embedding = nn.Linear(output_dim, d_model)
        self.pos_embedding = nn.Embedding(self.total_seq_len, d_model)
        self.transformer = nn.Transformer(d_model,
                                          nhead,
                                          num_encoder_layers,
                                          num_decoder_layers,
                                          dim_feedforward,
                                          dropout,
                                          activation)
        self.linear = nn.Linear(d_model, output_dim)
    
    def generate_square_subsequent_mask(size):
        """Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
            
        >>> generate_square_subsequent_mask(3)
        tensor([[-inf, -inf, -inf],
                [0., -inf, -inf],
                [0., 0., -inf]])
        """
        mask = torch.triu(torch.ones(size, size))
        mask = mask.float().masked_fill(mask == 1, float('-inf'))
        return mask.to(self.device)
        
    def forward(self, encoder_input, decoder_input):
        decoder_input_len = decoder_input.shape[0]
        decoder_mask = self.generate_square_subsequent_mask(decoder_input_len)
        # decoder_mask shape: (30, 30), 用来进行30次计算（可并行），每次计算用mask的一行
        
        encoder_input = self.encoder_embedding(encoder_input) * self.scale + \
                        self.pos_embedding(
                            torch.arange(0, self.source_seq_len, device=self.device)
                        ).unsqueeze(1)
        # encoder_input shape: (19, batch size, 512)
        
        decoder_input = self.decoder_embedding(decoder_input) * self.scale + \
                        self.pos_embedding(
                            torch.arange(self.source_seq_len,
                                         self.total_seq_len,
                                         device=self.device)
                        ).unsqueeze(1)
        # decoder_input shape: (30, batch size, 512)
        
        output = self.transformer(encoder_input, decoder_input, tgt_mask = decoder_mask)
        # output shape: (30, batch size, 512)
        
        return self.linear(output)
        # return tensor shape: (30, batch size, 2)

# 创建模型并初始化模型参数

In [94]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def initialize_weights(model):
    if hasattr(model, 'weight') and model.weight.dim() > 1:
        nn.init.xavier_uniform_(model.weight.data)
        
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = TrajectoryTransformer(device = dev).to(dev)
print(f'The model has {count_parameters(model):,} trainable parameters')
model.apply(initialize_weights)

The model has 44,169,730 trainable parameters


TrajectoryTransformer(
  (encoder_embedding): Linear(in_features=2, out_features=512, bias=True)
  (decoder_embedding): Linear(in_features=2, out_features=512, bias=True)
  (pos_embedding): Embedding(49, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): Multih